### This code is for pulling data from OSM (Openstreetmap) API, process, analyze, and mung the data to get to a form where it can be added to IBTTA's data warehouse. 

- The data being queried and pulled here is all the tolling facilities in countries all around the world, including facility names, geolocations, relation with adjacent ways and nodes
- In addition to the above, the code indirectly calculates the length of all ways (Center Lane Miles of all Toll Roads) through obtaining the coordinates of each node on the way. 
- Distances between individual nodes are calculated (with WSG84 projection) through GeoPy package and then rolled up to get the length of the entire toll road. All of these metrics are vital for the tolling industry.
- This codes starts with Mexico, and is optimzed to take any country as an input with minimal manual work.

In [1]:
# Connect to API, Pull Data

import overpy
import pandas as pd

api = overpy.Overpass()
result = api.query("""area[name="México"][boundary=administrative]->.searchArea;
// gather results
(
  // query part for: “toll=yes”
  node["toll"="yes"](area.searchArea);
  way["toll"="yes"](area.searchArea);
//  relation["toll"="yes"](area.searchArea);
);
// print results
out;
>;
out skel qt;""")

out_ways = []
for way in result.ways:
    out_ways.append(way)

out_relations = []
for relation in result.relations:
    out_relations.append(relation)

out_nodes = []
for node in result.nodes:
    out_nodes.append(node)

In [2]:
# Extract and check info

for way in result.ways:
    print("ID: %s" % way.id)
    print("Name: %s" % way.tags.get("name", "n/a"))
    print("  Highway: %s" % way.tags.get("highway", "n/a"))
    print("  Toll: %s" % way.tags.get("toll", "n/a"))
    print("  Ref: %s" % way.tags.get("ref", "n/a"))
    print("  Bridge: %s" % way.tags.get("bridge", "no"))
    print("  County: %s" % way.tags.get("tiger:county", "n/a"))
    print("  Name_Base: %s" % way.tags.get("tiger:name_base", "n/a"))
    print("  Name_Base1: %s" % way.tags.get("tiger:name_base_1", "n/a"))
    print("  Name_Base2: %s" % way.tags.get("tiger:name_base_2", "n/a"))
    print("  Name_Base_Type: %s" % way.tags.get("tiger:name_type", "n/a"))
    print("  Name_Base_Type: %s" % way.tags.get("tiger:name_type", "n/a"))

ID: 4596036
Name: Ramal Aeropuerto - Autopista
  Highway: motorway
  Toll: yes
  Ref: MEX 180
  Bridge: no
  County: n/a
  Name_Base: n/a
  Name_Base1: n/a
  Name_Base2: n/a
  Name_Base_Type: n/a
  Name_Base_Type: n/a
ID: 5035484
Name: Avenida Benito Juárez
  Highway: primary
  Toll: yes
  Ref: n/a
  Bridge: yes
  County: n/a
  Name_Base: n/a
  Name_Base1: n/a
  Name_Base2: n/a
  Name_Base_Type: n/a
  Name_Base_Type: n/a
ID: 14306388
Name: Autopista de Peaje Tijuana-Ensenada
  Highway: motorway
  Toll: yes
  Ref: MEX 1D
  Bridge: no
  County: n/a
  Name_Base: n/a
  Name_Base1: n/a
  Name_Base2: n/a
  Name_Base_Type: n/a
  Name_Base_Type: n/a
ID: 14879918
Name: Boulevard Lazaro Cardenas
  Highway: primary
  Toll: yes
  Ref: MEX 1;MEX 3
  Bridge: no
  County: n/a
  Name_Base: n/a
  Name_Base1: n/a
  Name_Base2: n/a
  Name_Base_Type: n/a
  Name_Base_Type: n/a
ID: 14882163
Name: Autopista de Peaje Tijuana-Ensenada
  Highway: motorway
  Toll: yes
  Ref: MEX 1D
  Bridge: no
  County: n/a
  N

In [3]:
# Extract Ways Only
import pandas as pd

WaysList = []
for way in result.ways:
    Way_ID = way.id
    WaysList.append(Way_ID)
Ways_df = pd.DataFrame(WaysList)
Ways_df.T.to_csv("Ways-2.csv", header=1)

In [4]:
# Extract Nodes Only

NodesList = []
for way in result.ways:
    Nodes_in_way = way.nodes
    NodesList.append(Nodes_in_way)
Nodes_df = pd.DataFrame(NodesList)
Nodes_df.T.to_csv("Nodes-2.csv", header=1)

In [5]:
# Test

df = pd.read_csv("Test.csv", header = 0)
df['4596036 Node_ID'], df['4596036 lat'], df['4596036 long']= df['4596036'].str.split(' ', 2).str
del df['4596036']
df

,4596036 Node_ID,4596036 lat,4596036 long
0,29053165,21.0186477,-86.8566685
1,3851407682,21.0187126,-86.8567567
2,867789063,21.0194475,-86.8576867
3,867789098,21.0215049,-86.8603235
4,29052413,21.0259660,-86.8662428
5,29052415,21.0273180,-86.8681340
6,29052416,21.0293356,-86.8714182
7,29052417,21.0340798,-86.8789642
8,29052418,21.0392850,-86.8872651
9,29052420,21.0464579,-86.8987707


In [7]:
# Generalize

df = pd.read_csv("Ways_w_Nodes-2.csv", header = 0)
for column in df.columns:

    df['{} Node_ID'.format(column)], df['{} lat'.format(column)], df['{} long'.format(column)] = df['{}'.format(column)].str.split(' ', 2).str
    del df['{}'.format(column)]
df.to_csv("Ways_w_Nodes-3.csv", header=1)

Id_cols = []
for i in range (0, len(df.columns)):
    if i % 3 == 0:
        Id_cols.append(i)
df_nocoord = df.iloc[:, Id_cols] 
df_nocoord.head(4)

# df is dataframe with nodes coordinates
# do_nocoord is dataframe without nodes coordinates

C:\Users\oaskar\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,

,4596036 Node_ID,5035484 Node_ID,14306388 Node_ID,14879918 Node_ID,14882163 Node_ID,15156348 Node_ID,15495772 Node_ID,17349353 Node_ID,17506921 Node_ID,24502373 Node_ID,...,558640844 Node_ID,558640845 Node_ID,560229783 Node_ID,560229788 Node_ID,560229789 Node_ID,560229991 Node_ID,560230532 Node_ID,560230730 Node_ID,560241399 Node_ID,560241400 Node_ID
0,29053165,33675708,680045094,1271633011,680045074,155095216,154476613,559483056,62786993,1505226927,...,5388189845,5388189844,299812186,4010947757,4010947777,4010934561,4010947748,4010947776,5004068273,5273564077
1,3851407682,559482867,103725506,4025122508,137484126,155095218,2726131853,33675709,62786996,1505226925,...,301121685,5388189845,299812187,299812186,4010947757,4010947776,4010947777,4010947748,5004068274,5004068278
2,867789063,NaN,1256569319,122129592,1256569622,1255197900,154477397,33675710,1061027217,1505226920,...,1746293812,NaN,299812190,NaN,NaN,NaN,NaN,NaN,NaN,5004068279
3,867789098,NaN,1256569387,147032616,1256569661,155095223,1253873909,33675711,62786998,266454887,...,NaN,NaN,1966778230,NaN,NaN,NaN,NaN,NaN,NaN,1228098340


In [8]:
# Identify which columns are for what (coordinates vs distances)

Coordinate_Cols = []
Length_cols = []
for i in range (0, len(df.columns)):
    if i % 3 == 0:
        Coordinate_Cols.append(df.columns[i])
        Length_cols.append(df.columns[i].replace('Node_ID', 'Length'))
Length_cols

# Fill in "Length" columns with any values for now
for i in range (0, len(Length_cols)):
    df['{}'.format(Length_cols[i])] = df['{}'.format(Coordinate_Cols[i])]
    
df.head(10)

,4596036 Node_ID,4596036 lat,4596036 long,5035484 Node_ID,5035484 lat,5035484 long,14306388 Node_ID,14306388 lat,14306388 long,14879918 Node_ID,...,558640844 Length,558640845 Length,560229783 Length,560229788 Length,560229789 Length,560229991 Length,560230532 Length,560230730 Length,560241399 Length,560241400 Length
0,29053165,21.0186477,-86.8566685,33675708,31.7459957,-106.4864653,680045094,32.2621566,-116.9899751,1271633011,...,5388189845,5388189844,299812186,4010947757,4010947777,4010934561,4010947748,4010947776,5004068273,5273564077
1,3851407682,21.0187126,-86.8567567,559482867,31.7461566,-106.4865224,103725506,32.2617268,-116.9881486,4025122508,...,301121685,5388189845,299812187,299812186,4010947757,4010947776,4010947777,4010947748,5004068274,5004068278
2,867789063,21.0194475,-86.8576867,NaN,NaN,NaN,1256569319,32.2615832,-116.9874731,122129592,...,1746293812,NaN,299812190,NaN,NaN,NaN,NaN,NaN,NaN,5004068279
3,867789098,21.0215049,-86.8603235,NaN,NaN,NaN,1256569387,32.2615105,-116.9869822,147032616,...,NaN,NaN,1966778230,NaN,NaN,NaN,NaN,NaN,NaN,1228098340
4,29052413,21.0259660,-86.8662428,NaN,NaN,NaN,1256569715,32.2614327,-116.9861476,1256569636,...,NaN,NaN,1966778216,NaN,NaN,NaN,NaN,NaN,NaN,5004068277
5,29052415,21.0273180,-86.8681340,NaN,NaN,NaN,1256569360,32.2607789,-116.9798640,1256569277,...,NaN,NaN,1221125115,NaN,NaN,NaN,NaN,NaN,NaN,1228098292
6,29052416,21.0293356,-86.8714182,NaN,NaN,NaN,122603155,32.2604490,-116.9766653,1271661936,...,NaN,NaN,299812191,NaN,NaN,NaN,NaN,NaN,NaN,5004068276
7,29052417,21.0340798,-86.8789642,NaN,NaN,NaN,1256569276,32.2603741,-116.9759981,147032624,...,NaN,NaN,1221125105,NaN,NaN,NaN,NaN,NaN,NaN,1228098170
8,29052418,21.0392850,-86.8872651,NaN,NaN,NaN,1256569635,32.2602911,-116.9754396,1256569599,...,NaN,NaN,1966778255,NaN,NaN,NaN,NaN,NaN,NaN,1228098511
9,29052420,21.0464579,-86.8987707,NaN,NaN,NaN,122603157,32.2601410,-116.9748742,1271662281,...,NaN,NaN,1221125132,NaN,NaN,NaN,NaN,NaN,NaN,5004068275


In [9]:
# Prepare df_nocoord for looping

Id_cols = []
for i in range (0, len(df.columns)):
    if i % 3 == 0:
        Id_cols.append(i)
df_nocoord = df.iloc[:, Id_cols] 
df_nocoord.head(20)

,4596036 Node_ID,5035484 Node_ID,14306388 Node_ID,14879918 Node_ID,14882163 Node_ID,15156348 Node_ID,15495772 Node_ID,17349353 Node_ID,17506921 Node_ID,24502373 Node_ID,...,554344017 Length,554345910 Length,554345917 Length,554361997 Length,554580694 Length,555492099 Length,555492110 Length,560229783 Length,560229991 Length,560241399 Length
0,29053165,33675708,680045094,1271633011,680045074,155095216,154476613,559483056,62786993,1505226927,...,5349905758,5349925384,5349925379,5350096053,5351902963,5359357542,5359357545,299812186,4010934561,5004068273
1,3851407682,559482867,103725506,4025122508,137484126,155095218,2726131853,33675709,62786996,1505226925,...,316308683,5349925378,5349925386,5350096054,329987849,1234971899,1797180852,299812187,4010947776,5004068274
2,867789063,NaN,1256569319,122129592,1256569622,1255197900,154477397,33675710,1061027217,1505226920,...,316308685,NaN,NaN,NaN,3752333751,1234971945,5359357544,299812190,NaN,NaN
3,867789098,NaN,1256569387,147032616,1256569661,155095223,1253873909,33675711,62786998,266454887,...,316308687,NaN,NaN,NaN,329987848,5359357538,NaN,1966778230,NaN,NaN
4,29052413,NaN,1256569715,1256569636,137484133,1255198054,154477398,33675712,NaN,266454889,...,316308688,NaN,NaN,NaN,329987847,1234971893,NaN,1966778216,NaN,NaN
5,29052415,NaN,1256569360,1256569277,137484135,155095225,1253873863,559480855,NaN,266454890,...,316308690,NaN,NaN,NaN,329987846,1234971885,NaN,1221125115,NaN,NaN
6,29052416,NaN,122603155,1271661936,1256569577,155095227,4845428443,179679997,NaN,266454892,...,316308691,NaN,NaN,NaN,301545630,1234971911,NaN,299812191,NaN,NaN
7,29052417,NaN,1256569276,147032624,1256569411,4078008304,1253873880,179680010,NaN,266454894,...,316308693,NaN,NaN,NaN,329987844,3420572016,NaN,1221125105,NaN,NaN
8,29052418,NaN,1256569635,1256569599,137484136,560216417,154477401,179680013,NaN,266454898,...,316308695,NaN,NaN,NaN,329987843,NaN,NaN,1966778255,NaN,NaN
9,29052420,NaN,122603157,1271662281,1256569309,155095228,154477402,559481025,NaN,266454901,...,316308696,NaN,NaN,NaN,329987842,NaN,NaN,1221125132,NaN,NaN


In [16]:
# Deal with NaN values

# df.fillna(value=0.00001, inplace=True)
for i in range (0, len(Length_cols)):
    df['{}'.format(Length_cols[i])][df['{}'.format(Length_cols[i])] != 0.00001] = 0.0

In [19]:
df[Length_cols].head(25)

,4596036 Length,5035484 Length,14306388 Length,14879918 Length,14882163 Length,15156348 Length,15495772 Length,17349353 Length,17506921 Length,24502373 Length,...,558640844 Length,558640845 Length,560229783 Length,560229788 Length,560229789 Length,560229991 Length,560230532 Length,560230730 Length,560241399 Length,560241400 Length
0,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.0,...,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.0,...,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
2,0.00000,0.00001,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.0,...,0.00000,0.00001,0.0,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00000
3,0.00000,0.00001,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.0,...,0.00001,0.00001,0.0,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00000
4,0.00000,0.00001,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.00001,0.0,...,0.00001,0.00001,0.0,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00000
5,0.00000,0.00001,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.00001,0.0,...,0.00001,0.00001,0.0,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00000
6,0.00000,0.00001,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.00001,0.0,...,0.00001,0.00001,0.0,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00000
7,0.00000,0.00001,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.00001,0.0,...,0.00001,0.00001,0.0,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00000
8,0.00000,0.00001,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.00001,0.0,...,0.00001,0.00001,0.0,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00000
9,0.00000,0.00001,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.00001,0.0,...,0.00001,0.00001,0.0,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00000


In [18]:
# Check Types (we need all floats to calculate distances)
for col in Length_cols:
    df.loc[:, col] = df.loc[:, col].astype(float)
type(df['5035484 Length'][3])

numpy.float64

In [39]:
# Calculate Distances
import geopy as gp
import math
from geopy.distance import vincenty

df_backup = df.copy()

Col_counter1 = 1
Col_counter2 = 2
for i in range (0, len(Length_cols)):#
#    gen = (x for x in range (0, len(df.iloc[:, 0])) if df.iloc[(x+2), Col_counter1] != 0)
    for x in range (0, (len(df.iloc[:, i])-1)):
        x_1 = x + 1
        if (df.iloc[x_1, Col_counter1] != 0.00001):
            
            lat1 = df.iloc[x, Col_counter1]
            long1 = df.iloc[x, Col_counter2]
            lat2 = df.iloc[x+1, Col_counter1]
            long2 = df.iloc[x+1, Col_counter2]

            distance = vincenty((long1, lat1), (long2, lat2)).miles
            # print(distance)
            # df['{}'.format(Length_cols[i])][x] = distance
            df.loc[x, ('{}'.format(Length_cols[i]))] = distance
        #print(df.iloc[x_2, Col_counter1])
        #print(lat1)
        #print(long1)
        #print(lat2)
        #print(long2)
        
    Col_counter1 = Col_counter1 + 3
    Col_counter2 = Col_counter2 + 3

In [13]:
# Test Distance
lat1 = 31.9032944
long1 = -116.7318013
lat2 = 31.9032528
long2 = -116.7320582


distance2 = vincenty((long1, lat1), (long2, lat2)).miles
distance2

0.017840598664411806

In [40]:
# Ways lengths
Lengths_List = []

for i in range(0, len(Length_cols)):
    l = len(df['{}'.format(Length_cols[i])][df['{}'.format(Length_cols[i])] != 0.00001])
    #print(l)
    df['{}'.format(Length_cols[i])][l] = df['{}'.format(Length_cols[i])][0:l-1].sum()
    Lengths_List.append(df['{}'.format(Length_cols[i])][l])

C:\Users\oaskar\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [41]:
df[Length_cols].head(22)

,4596036 Length,5035484 Length,14306388 Length,14879918 Length,14882163 Length,15156348 Length,15495772 Length,17349353 Length,17506921 Length,24502373 Length,...,558640844 Length,558640845 Length,560229783 Length,560229788 Length,560229789 Length,560229991 Length,560230532 Length,560230730 Length,560241399 Length,560241400 Length
0,0.006126,0.005071,0.127223,0.021508,0.078046,0.099288,0.009681,0.002418,0.131292,0.050480,...,0.057222,0.017172,0.056489,0.199914,0.02369,0.146587,0.062412,0.052532,0.028525,0.314015
1,0.064604,0.000000,0.047002,0.029005,0.067966,0.028823,0.014304,0.014014,0.060728,0.031219,...,0.077201,0.000000,1.055666,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.062337
2,0.183165,0.005071,0.034076,0.092520,0.138521,0.024448,0.018137,0.009658,0.012438,0.435301,...,0.000000,0.017172,0.030383,0.199914,0.02369,0.146587,0.062412,0.052532,0.028525,0.056375
3,0.411156,0.000010,0.057856,0.020268,0.210671,0.020008,0.022958,0.005624,0.000000,0.066111,...,0.134423,0.000010,0.029635,0.000010,0.00001,0.000010,0.000010,0.000010,0.000010,0.041707
4,0.131352,0.000010,0.435683,0.019782,0.029180,0.020698,0.010213,0.008826,0.204458,0.124702,...,0.000010,0.000010,0.026684,0.000010,0.00001,0.000010,0.000010,0.000010,0.000010,0.037050
5,0.228056,0.000010,0.221783,0.105049,0.020280,0.064155,0.003533,0.007202,0.000010,0.119419,...,0.000010,0.000010,0.098475,0.000010,0.00001,0.000010,0.000010,0.000010,0.000010,0.027713
6,0.524010,0.000010,0.046270,0.086057,0.016523,0.025213,0.003371,0.011485,0.000010,0.153873,...,0.000010,0.000010,0.029459,0.000010,0.00001,0.000010,0.000010,0.000010,0.000010,0.023086
7,0.576428,0.000010,0.038769,0.023346,0.016666,0.045251,0.013513,0.012219,0.000010,0.134833,...,0.000010,0.000010,0.032390,0.000010,0.00001,0.000010,0.000010,0.000010,0.000010,0.048825
8,0.798960,0.000010,0.039443,0.019275,0.043920,0.044023,0.006231,0.009392,0.000010,0.113024,...,0.000010,0.000010,0.039661,0.000010,0.00001,0.000010,0.000010,0.000010,0.000010,0.359641
9,0.567403,0.000010,0.037750,0.018402,0.080265,0.012273,0.008460,0.000000,0.000010,0.913152,...,0.000010,0.000010,0.037113,0.000010,0.00001,0.000010,0.000010,0.000010,0.000010,0.047588


In [43]:
len(Lengths_List)

4373

In [44]:
len(Length_cols)

4373

### They match!

In [46]:
# Dictionary of all "Ways" and Lengths
Lengths_dict = [{Length_cols[i] : Lengths_List[i]} for i in range (0, len(Length_cols) -1)]
print(Lengths_dict)

[{'4596036 Length': 4.9855012977049871}, {'5035484 Length': 0.0050711851909326471}, {'14306388 Length': 2.9655818288542117}, {'14879918 Length': 0.47821376737089977}, {'14882163 Length': 0.85874267955248351}, {'15156348 Length': 1.725577375304109}, {'15495772 Length': 0.11040325023038697}, {'17349353 Length': 0.080838436139025763}, {'17506921 Length': 0.20445832210817777}, {'24502373 Length': 277.87275008728096}, {'24907917 Length': 0.042715913436210472}, {'24907918 Length': 0.024955363958472289}, {'24907928 Length': 0.0808169418080429}, {'24907934 Length': 0.020158768967622691}, {'24907959 Length': 0.087722320214486102}, {'24907963 Length': 0.0099508707322209389}, {'24907967 Length': 0.01064206899319867}, {'24924170 Length': 0.0091427686458281331}, {'24924323 Length': 0.0078497417234863528}, {'24924324 Length': 0.028946690337726448}, {'24924328 Length': 0.0089775359872612502}, {'24924338 Length': 0.12614413244611686}, {'24924343 Length': 0.078732928307900185}, {'24924429 Length': 0.01

In [72]:
# To CSV
df.to_csv("df_w_distance.csv", header = 1)
df[Length_cols].to_csv("df_Length_cols_only.csv", header = 1)

In [71]:
del Ways_w_Lengths

Ways_w_Lengths = pd.DataFrame(Length_cols)
Ways_w_Lengths['Length'] = pd.Series(Lengths_List)
Ways_w_Lengths.columns = ['Way', 'Length']
Ways_w_Lengths['Way'] = Ways_w_Lengths['Way'].map(lambda x: str(x)[:-7])
Ways_w_Lengths


,Way,Length
0,4596036,4.985501
1,5035484,0.005071
2,14306388,2.965582
3,14879918,0.478214
4,14882163,0.858743
5,15156348,1.725577
6,15495772,0.110403
7,17349353,0.080838
8,17506921,0.204458
9,24502373,277.872750


In [ ]:
# Deal with Nodes in common between two Ways

all_data_tags = all_data.copy()

x = 0
for i in range (0, len(df_nocoord.columns)):
    for j in range (x, x+512):
        if all_data.iloc[j, 0] != 'NaN':
            all_data_tags.iloc[j, 0] = '{}'.format(df_nocoord.columns[i])
        else:
            pass;
    x= x + 512;

all_data_tags.to_csv("Labels.csv")
all_data_tags['Dup_Value'] = np.zeros(all_data_tags.shape[0])

In [ ]:
All_df = pd.concat([all_data, all_data_tags], axis=1)
All_df.columns = ['Node', 'Way', 'Dup_Value'] 
All_df.dropna(axis=0, how='any', inplace=True)

All_df['Dup_Value'] = np.zeros(All_df.shape[0])
count = 1
for i in range (0, All_df.shape[0]):
    x = All_df.iloc[i, 0]
    for j in range (i+1, All_df.shape[0]):
        if All_df.iloc[(j), 0] == x:
            All_df.iloc[i, 2] = count
            All_df.iloc[j, 2] = count
            count=count+1
All_df.to_csv("All_df.csv", header = 1)

In [15]:
df_nocoord['is_duplicated'] = df_nocoord.duplicated(['4596036 Node_ID', '5035484 Node_ID'])
df_nocoord['is_duplicated']

C:\Users\oaskar\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


0      False
1      False
2      False
3      False
4      False
5      False
6      False
7      False
8      False
9      False
10     False
11     False
12     False
13     False
14     False
15     False
16     False
17     False
18     False
19     False
20     False
21      True
22      True
23      True
24      True
25      True
26      True
27      True
28      True
29      True
       ...  
482     True
483     True
484     True
485     True
486     True
487     True
488     True
489     True
490     True
491     True
492     True
493     True
494     True
495     True
496     True
497     True
498     True
499     True
500     True
501     True
502     True
503     True
504     True
505     True
506     True
507     True
508     True
509     True
510     True
511     True
Name: is_duplicated, Length: 512, dtype: bool

In [ ]:
# Append Results to Lists
Ways_CA = []
for way in result.ways:
    Ways_CA.append(["Way ID ", way.id])
    Ways_CA.append(["Highway ", way.tags.get("highway", "n/a")])
    Ways_CA.append(["Toll ", way.tags.get("toll", "n/a")])
    Ways_CA.append(["Reference ", way.tags.get("ref", "n/a")])
    Ways_CA.append(["Bridge ", way.tags.get("bridge", "n/a")])
    Ways_CA.append(["County ", way.tags.get("tiger:county", "n/a")])
    Ways_CA.append(["Name ", way.tags.get("name", "n/a")])
    Ways_CA.append(["Name2 ", way.tags.get("tiger:name_base", "n/a")])
    Ways_CA.append(["Name3 ", way.tags.get("tiger:name_base_1", "n/a")])
    Ways_CA.append(["Name4 ", way.tags.get("tiger:name_base_2", "n/a")])
    Ways_CA.append(["Name Type ", way.tags.get("tiger:name_type", "n/a")])

Relation_CA = []
for relation in result.ways:
    Relation_CA.append(["Relation ID ", relation.id])
    Relation_CA.append(["Highway ", relation.tags.get("highway", "n/a")])
    Relation_CA.append(["Toll ",relation.tags.get("toll", "n/a")])
    Relation_CA.append(["Reference ", relation.tags.get("ref", "n/a")])
    Relation_CA.append(["Bridge ", relation.tags.get("bridge", "n/a")])
    Relation_CA.append(["County ", relation.tags.get("tiger:county", "n/a")])
    Relation_CA.append(["Name ", relation.tags.get("name", "n/a")])
    Relation_CA.append(["Name2 ", relation.tags.get("tiger:name_base", "n/a")])
    Relation_CA.append(["Name3 ", relation.tags.get("tiger:name_base_1", "n/a")])
    Relation_CA.append(["Name4 ", relation.tags.get("tiger:name_base_2", "n/a")])
    Relation_CA.append(["Name Type ", relation.tags.get("tiger:name_type", "n/a")])

Nodes_CA = []
for node in result.nodes:
    Nodes_CA.append(["Node ID ", node.id])
    Nodes_CA.append(["Highnode ", node.tags.get("highnode", "n/a")])
    Nodes_CA.append(["Toll ", node.tags.get("toll", "n/a")])
    Nodes_CA.append(["Reference ", node.tags.get("ref", "n/a")])
    Nodes_CA.append(["Bridge ", node.tags.get("bridge", "n/a")])
    Nodes_CA.append(["County ", node.tags.get("tiger:county", "n/a")])
    Nodes_CA.append(["Name ", node.tags.get("name", "n/a")])
    Nodes_CA.append(["Name2 ", node.tags.get("tiger:name_base", "n/a")])
    Nodes_CA.append(["Name3 ", node.tags.get("tiger:name_base_1", "n/a")])
    Nodes_CA.append(["Name4 ", node.tags.get("tiger:name_base_2", "n/a")])
    Nodes_CA.append(["Name Type ", node.tags.get("tiger:name_type", "n/a")])


In [ ]:
# Check

print(Nodes_CA)
print(Relation_CA)
print(Ways_CA)

In [ ]:
# Ways to CSV
#ListByFeature = Ways_CA_Df.groupby('Feature')['Value'].apply(list)
ListByFeature = Ways_CA_Df.groupby('Feature')['Value'].apply(lambda x: pd.Series(x.tolist())).unstack().add_prefix('Value_')
ListByFeature.to_csv("Ways_MexicoNew.csv", header=1)

In [ ]:
# Nodes to CSV
ListByFeature = nodes_CA_Df.groupby('Feature')['Value'].apply(list)
type(ListByFeature)
DFByFeature = pd.DataFrame(ListByFeature)
df = DFByFeature.Value
df = pd.DataFrame(df)
df.to_csv("Nodes.csv", header=0)